In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
number_of_types = 5

In [3]:
columns = ['Состояние', 'Qж, м3/сут', 'Qж ТМ, м3/сут', 'Qн, т/сут',
       'Qн ТМ, т/сут', 'ГФ, м3/т', 'ГФ ТМ, м3/т', 'Обв, %', 'Обв ХАЛ, %',
       'Нд, м', 'Рбуф, атм', 'Рлин, атм', 'Рзатр, атм', 'Dшт, мм', 'КВЧ',
       'Прим', 'I, А', 'F, Гц', 'Рприем, атм', 'Рзаб, атм', 'Рзаб зам',
       'Мехпримеси (ХАЛ), мг/дм3']

In [4]:
data_paths1 = [
    'data\\КП 1, 1бис, 2, 2бис\\1. Новая книга (01.01.2018-01.01.2019).xls', 
    'data\\КП 1, 1бис, 2, 2бис\\2. Новая книга (01.01.2019-01.01.2020).xls', 
    'data\\КП 1, 1бис, 2, 2бис\\3. Новая книга (01.01.2020-12.01.2021).xls',
]

In [5]:
data_paths2 = [
    'data\\КП 3,3б,4,4б,5,5б,6,6б\\1. Новая книга (01.01.2018-01.01.2019).xls', 
    'data\\КП 3,3б,4,4б,5,5б,6,6б\\2. Новая книга (01.01.2019-01.01.2020).xls', 
    'data\\КП 3,3б,4,4б,5,5б,6,6б\\3. Новая книга (01.01.2020-14.01.2021).xls', 
]

In [6]:
data_paths3 = [
    'data\\КП 7, 8, 10\\7,8,10_2018.xls' ,
    'data\\КП 7, 8, 10\\7,8,10_2019.xls' ,
    'data\\КП 7, 8, 10\\7,8,10_2020.xls' ,
]

In [7]:
break_paths = [
    'data\\Аварии\\НЭК.xlsx', 
    'data\\Аварии\\Отказы (газ, Рпл).xlsx', 
    'data\\Аварии\\Отказы (ГНО).xlsx', 
    'data\\Аварии\\Отказы (мех.примеси).xlsx', 
    'data\\Аварии\\Отказы (прочее).xlsx', 
]

In [8]:
# вспомогательная функция для именования колонок
def create_dict():
    d = dict()
    global columns
    i = 1
    for col in columns:
        d[i] = col
        i += 1
    return d

In [9]:
# вспомогательная функция для именования колонок
def something(x):
    d = create_dict()
    return d[x % len(columns) + 1]

In [10]:
# вспомогательная функция для нумерации
bar = -1
def foo(x):
    global bar
    bar += 1
    return bar

### Считывание одного файла

In [11]:
def read_one(path, number_of_types, sheet_no):
    data = pd.read_excel(io=path, sheet_name=sheet_no)
    data = data.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1)
    
    # преобразование даты
    date = data.iloc[0][2:]
    for i in range(len(date.values)):
        if type(date[date.index[i]]) == datetime.datetime:
            date[date.index[i]] = datetime.datetime.date(date[date.index[i]])
    data = data.drop('Парам/Дата', axis=1)
    data = data.drop(0)
    
    
    i = 1
    s = data['Скв.'].iloc[0] # номер скважины
    s = s[:s.index(' ')]
    frames = dict() # Ключ - номер скважины; Значения - данные по скважине
    frames[s] = pd.DataFrame() 
    frames[s] = frames[s].append(pd.Series(data.iloc[0].values), ignore_index=True)
    while True:
        # Добавление строк в количестве колонок
        for _ in range(len(columns) -1): 
            frames[s] = frames[s].append(pd.Series(data.iloc[i].values), ignore_index=True)
            i += 1
            if i >= len(data):
                break
        if i >= len(data):
            break
            
        # Смена скважины 
        s = data['Скв.'].iloc[i]
        s = s[:s.index(' ')]
        if s not in frames:
            frames[s] = pd.DataFrame()
            
        frames[s] = frames[s].append(pd.Series(data.iloc[i].values), ignore_index=True)
        i += 1
    
    for key in frames.keys():
        frames[key] = frames[key].drop(0, axis=1)
        i = -1
        bar = -1
        frames[key] = frames[key].rename(index=something, columns=foo).transpose() # Траноспонирование таблицы
        frames[key]['Дата'] = date.values # Добавление даты 
        frames[key]['Тип Аварии'] = np.zeros(len(frames[key]))
        frames[key] = fill_breaks(frames[key], key, number_of_types) # заполнение информации об авариях
    
    return frames

### Сопоставление аварий

In [12]:
def fill_breaks(data, squajina, number_of_types):
    breaks_types = range(1, number_of_types + 1) 
    for breack_type in breaks_types: # Для каждого файла
        breaks = pd.read_excel(io=break_paths[breack_type - 1], usecols='C, D')
        breaks = breaks.drop(0)
        for br in breaks.values:
            if str(br[0]) == squajina: 
                # Если та же скважина, то везде, где дата совпадает, присвоить значение аварии
                data.loc[data['Дата'] == datetime.datetime.date(br[1])]['Тип Аварии'] = breack_type
    return data

In [13]:
paths = [data_paths1, data_paths2, data_paths3]
total_frames = []
n = -1
for d_paths in paths:
    print(d_paths[0])
    total_frames.append(read_one(d_paths[0], number_of_types, 0)) # Записываем первый файл
    n += 1
    for i in range(1, len(d_paths)): # Добавляем два других
        print(d_paths[i])
        tmp_frame = read_one(d_paths[i], number_of_types, 0)
        for key in total_frames[n].keys():
            # В каждую таблицу с данными из первого файла добавить данные из соответствующих данных из других файлов
            total_frames[n][key] = total_frames[n][key].append(tmp_frame[key]) 

data\КП 1, 1бис, 2, 2бис\1. Новая книга (01.01.2018-01.01.2019).xls


<ipython-input-12-b2513cfdb7dd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Тип Аварии'][data['Дата'] == datetime.datetime.date(br[1])] = breack_type


data\КП 1, 1бис, 2, 2бис\2. Новая книга (01.01.2019-01.01.2020).xls


KeyboardInterrupt: 

In [ ]:
# Добавить к ним данные со второй страницы
n = -1
for d_paths in paths:
    n += 1
    for i in range(0, len(d_paths)):
        print(d_paths[i])
        tmp_frame = read_one(d_paths[i], number_of_types, 1)
        for key in total_frames[n].keys():
            total_frames[n][key] = total_frames[n][key].append(tmp_frame[key])

In [14]:
for frames in total_frames:
    for key in frames.keys():
        frames[key] = frames[key].reset_index()
        frames[key] = frames[key].drop('index', axis=1)

In [ ]:
total_frames[0].update(total_frames[1])
total_frames[0].update(total_frames[2])

### Запись данных в индивидуальный файл

In [17]:
for key in total_frames[0].keys():
    total_frames[0][key].to_csv('cooked data\\' + key + '.csv')

In [18]:
pd.DataFrame(total_frames[0].keys()).to_csv('cooked data\\' + '_squajins' + '.csv')